This notebook will be used to show some functions in ellipse.py and maximize the accessibility of this module.

#Using the ellipse.py Module
The velocities have to be prepared and set up in a certain way before they can be put through the tidal analysis. They need to be orthogonal, rotated to align with N/S, unstaggered and masked.

Creating tidal ellipses :

* From nowcast files
* From any other source
* From a source with different nodal correction values

In [1]:
import os
import datetime
import matplotlib.pylab as plt
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import seaborn as sns
from dateutil import tz
from scipy.optimize import curve_fit
import collections

import netCDF4 as nc
from salishsea_tools import (viz_tools, tidetools as tt, ellipse, nc_tools)
from salishsea_tools.nowcast import (analyze, research_VENUS)


%matplotlib inline

##Case 1: Nowcast date range

In [3]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'


#The longer the date range the longer the analysis takes but the more tidal constituents it can resolve
to=datetime.datetime(2015,7,10)
tf=datetime.datetime(2015,7,11)

i = [280]
j = [400]

ellipse_files_nowcast: finds all the u and v grid files within the date range in the path designated, organises them into chronological order and extracts the locations you have chosen.

Notes:

* The to and tf must be datetime.datetime objects
* The i and j must be in a list, the functions adds on i-1 and j-1 to the list in order to be able to unstagger and maintain the desired area or single location. Therefore 0 cannot be included in i or j
* Depthrange is the depth in METERS that you want to look at, for a single value the functions looks for the depth LESS THAN the value you gave, for a range the function will find values WITHIN this range

In [ ]:
# The function will extra the velocities at the bin nearest bin that is less than 5m deep.
u, v, time, dep = ellipse.ellipse_files_nowcast(to, tf, i, j, path, depthrange=5)

prepare_vel: Will do the unstaggering, rotating and depth averaging of the velocity time series.

Notes:

* If depav = True then there must be a depth vector that is associated with the depth bins of the velocities as the dpeth variable and there must be a more than one depth in the velocity vectors.

In [9]:
u_u, v_v = ellipse.prepare_vel(u, v, depav=False, depth='None')

get_param: This will do the actual fit and return a dictionary object of the constituents with and their Semi-Major Axis, Semi-Minor Axis, Inclination and Phase.

The constuents are in order of highest importance in the domain and come in pairs. 

* 2 constituents - M2, K1
* 4 constituents - M2, K1, S2, O1
* 6 constituents - M2, K1, S2, O1, N2, P1
* 8 constituents - M2, K1, S2, O1, N2, P1, K2, Q1

In [15]:
params = ellipse.get_params(u_u, v_v, time, 4)
print params['S2']

{'Phase': array([[ 359.05007629]]), 'Semi-Major Axis': array([[ 0.08765313]]), 'Semi-Minor Axis': array([[ 0.00176911]]), 'Inclination': array([[ 135.0407882]])}


#Case 2: Any other type of files

To show how to use the ellipse functions in any other type of file I will use he quarter-hourly files output from the locations of the Central and East node

In [2]:
grid = '/ocean/mdunn/MEOPAR/jun_VENUS_east_gridded.nc'
G = nc.Dataset(grid)
u = G.variables['vozocrtx'][:]
v = G.variables['vomecrty'][:]
dep = G.variables['depthu'][:]

#Set up the time array that corresponds to the times of the u and v arrays in hours
t = G.variables['time_counter'][:]/3600.

When analyzing for 6 or 8 tidal constituents you need a longer time series, athere will be an error if it is much too short.

The u and v arrays always have to be set up as [time, depth, x, y]. If there is only one depth [time, x, y] or one location [time, depth] or one depth and one location [time]

In [21]:
u_u,v_v = ellipse.prepare_vel(u, v, depav=True, depth=dep)

params = ellipse.get_params(u_u,v_v,t, 6)
print params['K1']['Phase']

[[ 251.18831966]]


##Case 3: The nodal correction dictionary

The functions assume you will be using the nodal corrections that are set by the nowcast. If this is not the case you have to set your own dictionary object with the correct values in it.

In [5]:
NodalCorr = {
    'K1': {
        'freq': 30.082138,
        'ft': 0.763545,
        'uvt': 344.740346},
    'M2': {
        'freq': 28.984106,
        'ft': 1.035390,
        'uvt': 346.114490}}

In [8]:
u_u,v_v = ellipse.prepare_vel(u, v, depav=True, depth=dep)
# In ellipse param set the tidecorr to be the correct dictionary.
params = ellipse.get_params(u_u,v_v,time, 2, tidecorr=NodalCorr)

In [9]:
print params['K1']

{'Phase': array([[ 284.96515857]]), 'Semi-Major Axis': array([[ 0.34321001]]), 'Semi-Minor Axis': array([[-0.03599698]]), 'Inclination': array([[ 128.21677008]])}


## Plotting
The functions to plot the ellipses are going to vary alot based on the area, constituent, amount, and the purpose of the figure.
There are many examples in [CODAREllips](http://nbviewer.ipython.org/urls/bitbucket.org/salishsea/analysis/raw/tip/Muriel/CODAREllipse.ipynb) and in [TidalEllipsesComparison](http://nbviewer.ipython.org/urls/bitbucket.org/salishsea/analysis/raw/tip/Muriel/TidalEllipsesComparisons.ipynb)
